# Loading and modifying a SMIRNOFF forcefield

This notebook illustrates how to load a SMIRNOFF forcefield, apply it to an example molecule, get the energy, then manipulate the parameters in the forcefield and update the energy.

## Prep some utility functions/import stuff

In [2]:
# Imports needed
from openforcefield.topology import Molecule, Topology
from openforcefield.typing.engines.smirnoff.forcefield import ForceField
from openforcefield.utils import get_data_filename
from simtk import openmm, unit
import numpy as np

In [6]:
# Define utility function we'll use to get energy of an OpenMM system
def get_energy(system, positions):
    """
    Return the potential energy.

    Parameters
    ----------
    system : simtk.openmm.System
        The system to check
    positions : simtk.unit.Quantity of dimension (natoms,3) with units of length
        The positions to use
    Returns
    ---------
    energy
    """

    integrator = openmm.VerletIntegrator(1.0 * unit.femtoseconds)
    context = openmm.Context(system, integrator)
    context.setPositions(positions)
    state = context.getState(getEnergy=True)
    energy = state.getPotentialEnergy() / unit.kilocalories_per_mole
    return energy

## Load an molecule and evaluate its energy before and after a parameter modification

In [3]:
# Load a molecule
from openforcefield.tests.utils import get_alkethoh_filepath
molecule = Molecule.from_file(get_alkethoh_filepath('AlkEthOH_c100'))

#Get positions for use below
positions = molecule.positions

# Load forcefield file
ffxml = get_data_filename('forcefield/Frosst_AlkEthOH.offxml')
ff = ForceField(ffxml)

# Generate an openforcefield Topology containing only this molecule
topology = molecule.to_topology()

# Create initial system
old_system = ff.create_system(topology)

# Get initial energy before parameter modification
old_energy = get_energy(old_system, positions)

# Get params for an angle
smirks = '[a,A:1]-[#6X4:2]-[a,A:3]' # SMIRKS for the parameter to retrieve
parameter = forcefield.forces['Angles'].parameters[smirks]

# Modify the parameters
parameter.k *= 0.9
parameter.theta0 *= 1.02

# Evaluate energy after parameter modification
new_system = ff.create_system(topology)
new_energy = get_energy(new_system, positions)

# Print out energy
print("Original energy: %.3g. New energy: %.3g" % (old_energy, new_energy))

UnboundLocalError: local variable 'mol' referenced before assignment

## Inspect and manipulate parameters

In [4]:
# Create a new ForceField containing the smirnoff99Frosst parameter set:

forcefield = ForceField('smirnoff99Frosst.offxml')

# Create an OpenMM system from a :class:`openforcefield.topology.Topology` object:

from openforcefield.topology.testsystems import WaterBox
system = forcefield.create_system(WaterBox.topology)

# Modify the long-range electrostatics method:

forcefield.forces['Electrostatics'].method = 'PME'

# Inspect the first few vdW parameters:

print(forcefield.forces['vdW'].parameters[0:3])

# Retrieve the vdW parameters by SMIRKS string and manipulate it:

parameter = forcefield.forces['vdW'].parameters['[#1:1]-[#7]']
parameter.sigma += 0.1 * unit.angstroms
parameter.epsilon *= 1.02

# Make a child vdW type more specific (checking modified SMIRKS for validity):

forcefield.forces['vdW'].parameters[-1].smirks += '$(*~[#53])'

NameError: name 'ParameterList' is not defined